# Ismétlés.

Előző órán:

- osztály, `class`

- dunder metódusok: `__add__`, `__mul__`, és társaik, pl. `__call__`. `Polinom` osztály műveleteit defináltuk így. 

- File műveletek. Írás olvasás fileból.
  Új  nyelvi konstrukció:

  ```python
  with open("./data/P0022_names.txt", "r") as f:
      text = f.read()
  ```

In [1]:
import ipytest
ipytest.autoconfig()

def print_lines(names, max_lines, per_line):
    for i in range(0, len(names), per_line):
        print(", ".join(names[i: i+per_line]), end="")
        if i == (max_lines-1)*per_line:
            if i < len(names):
                print(", ...")
            else:
                print()
            break
        print()
    
def print_dunder(cls, max_lines=float("inf"), per_line=8):
    print_lines([attr for attr in dir(cls) if attr.startswith("__")], max_lines, per_line)
    
def print_methods(cls, max_lines=float("inf"), per_line=8):
    print_lines([attr for attr in dir(cls) if not attr.startswith("_")], max_lines, per_line)

## Mitől lesz egy file (open ereménye) a `with` után használható? 

In [4]:
filepath = "./data/P0022_names.txt"
with open(filepath) as f:
    pass
print_dunder(f, 2, 5)

__class__, __del__, __delattr__, __dict__, __dir__
__doc__, __enter__, __eq__, __exit__, __format__, ...


<!-- Az `open` függvény azért használható a `with` statement-tel, mert van egy
 -->
A magyarázat kulcsa az `__enter__` és az `__exit__` metódus. Azokat az objektumokat, amik ezekkel rendelkeznek
`contextmanager`-nek is hívják.

Mi történik a következő kódrészlet végrehajtásakor?
```python
with obj as x:
    pass # do something with x
```

1. `x = obj.__enter__()`.  Ezen a ponton az `obj` objektum feljegyezheti, mit akar visszaállítani ha a `with` blokk véget ért. 

2. Törzs kiértékelése. Ez a példában `pass` nem csinál semmit.

3. `obj.__exit__(...)` végrehajtása. Itt a `...` paraméterek azt írják le, hogy sikerült-e a `with` blokk törzsét végrehajtani.

### Egyszerű `contextmanager` a működés szemléltetésére

In [5]:
class SimpleWith:
    def __init__(self):
        print("init")
    def __enter__(self):
        print("enter")
        return self

    def __exit__(self, exc_type, exc_value, traceback): # lehetne def __exit__(self, *args):
        print(f"exit called with {exc_type=}, {exc_value=}, {traceback=}")
        return True

with SimpleWith() as sw:
    print(f"inside {sw=}")

print("outside")

init
enter
inside sw=<__main__.SimpleWith object at 0x72a041186c50>
exit called with exc_type=None, exc_value=None, traceback=None
outside


## `with` statement-tel használható timer

In [6]:
import time

class Timer:
    def __init__(self):
        self._start = None
        self._elapsed_time = None

    def __enter__(self):
        self._start = time.perf_counter_ns()
        self._elapsed_time = None
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._elapsed_time = time.perf_counter_ns() - self._start
        print(f"elapsed_time {self.elapsed_time:.2f} ms")

    @property
    def elapsed_time(self):
        return self._elapsed_time/10**6 if self._elapsed_time else None

In [8]:
with Timer() as timer:
    time.sleep(1)

with timer:
    pass
#     time.sleep(0.001) 

elapsed_time 1001.32 ms
elapsed_time 0.00 ms


**HF** Írjunk `__str__` metódust a `Timer` osztályhoz, ami ,,kulturált'' formában kiírja az utolsónak mért futási időt.

# Kivételkezelés (Exception handling)

In [9]:
1 / 0

ZeroDivisionError: division by zero

In [10]:
a + 10

NameError: name 'a' is not defined

In [11]:
int("12.345")

ValueError: invalid literal for int() with base 10: '12.345'

Kivételek mindig is előfordulhatnak, azaz olyan helyzetek, amikor a számolás értelmetlen, vagy egy fájlt kell megnyitni, ami nem is létezik, egy adatbázishoz kapcsolódunk, ahol nem jó a jelszó, vagy nem elérhető a szerver, valamit konvertálni kell valami mássá, de nem lehetséges.

Pl. konvertáljuk egy sztringet egész számmá, ha ez lehetséges.

In [13]:
def convert_to_int(s):
    return int(s)


print(convert_to_int("100"))

convert_to_int("10.0")

100


ValueError: invalid literal for int() with base 10: '10.0'

Hogyan lehetünk biztosak abban, hogy egy stringet egész számként tudunk értelmezni?

In [14]:
def convert_to_int(string):
    if isinstance(string, str) and string.isnumeric():
        return int(string)
    
    return None


print(convert_to_int("123"))
print(convert_to_int("45.67"))
print(convert_to_int("-2"))

123
None
None


Mi lenne, ha nem mi próbálnánk kitalálni, hogy mi lenne a jó, hanem hagynánk, hogy a Python `int` konverziós függvénye *megpróbálja* a konverziót, aztán ha sikerül, akkor jó, ha meg nem, akkor meg kitalálunk valamit.

In [17]:
def convert_to_int(s):
    try:
        return int(s)
    except Exception as e:
        print(f"Cannot convert '{e}' to an integer.")
    
    return None


print(convert_to_int("100"))
print(convert_to_int("10.0"))
print(convert_to_int("-3"))

100
Cannot convert 'invalid literal for int() with base 10: '10.0'' to an integer.
None
-3


Később látni fogjuk, hogy nem csak egyfajta kivételt kell / lehet lekezelni, hanem akár többféle hibaok is előfordulhat. 

Egy tipikus ilyen helyzet a fájlbeolvasás, ahol lehet, hogy a 
* fájl nem található
* nincs jogunk olvasni a fájlt
* hibás a formátuma
* nem dekódolható a szöveg, mert más karakterkészlet szerepel benne, mint amire számítunk
* stb.

A kivételek kezelésének / elkapásának általános szintaxisa a `try`-`except` konstrukció:

```python
try:
    try_something()
except SomeError_1:
    do_something_1()
...
except SomeError_n:
    do_something_n()
else:
    do_this_when_there_was_no_exception()
finally:
    execute_anyway()
```

Az előforduló hibát akár újra is lehet dobni azért, hogy a minket hívó függvényhez is eljusson a hiba

In [19]:
def convert_to_int(s):
    try:
        return int(s) 
    except ValueError:
        print(f"Cannot convert {s} to an integer.")
        raise 
    finally:
        print("finally")
        
        
convert_to_int("12.34")

Cannot convert 12.34 to an integer.
finally


ValueError: invalid literal for int() with base 10: '12.34'

Vagy dobhatunk saját magunk által definiált hibát is:

In [21]:
class MyError(Exception):
    pass

def convert_to_int(s):
    try:
        return int(s)
    except ValueError:
        print(f"Cannot convert {s} to an integer.")
        raise TypeError("Catch me if you can.")
         
result = convert_to_int("12.34")

Cannot convert 12.34 to an integer.


TypeError: Catch me if you can.

In [22]:
def division(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero error!")
    else:
        print(f"The result is {result}")
    finally:
        print("This will execute anyway")

In [23]:
division(1, 2)

The result is 0.5
This will execute anyway


In [25]:
division(1, 0)

division by zero error!
This will execute anyway


# Kivétel `with` blokkon belül

In [26]:
class SimpleWith:
    def __init__(self, handle_exception=True):
        self.handle_exception = handle_exception

    def __enter__(self):
        print("enter")
        return self

    def __exit__(self, exc_type, exc_value, traceback): # lehetne def __exit__(self, *args):
        print(f"exit called with\n\t{exc_type=}\n\t{exc_value=}\n\t{traceback=}")
        if exc_type is not None and self.handle_exception:
            print(f"{exc_value!r} handled")
        return self.handle_exception

In [28]:
with SimpleWith(False) as sw:
    print(f"inside {sw=}")
    raise ValueError("from with block")

print("outside")

enter
inside sw=<__main__.SimpleWith object at 0x72a04055fcd0>
exit called with
	exc_type=<class 'ValueError'>
	exc_value=ValueError('from with block')
	traceback=<traceback object at 0x72a040432b40>


ValueError: from with block

# Polinom osztály `__init__` művelet ellenőrzéssel.

In [29]:
class Polinom:
    def __init__(self, *coeff):
        """
        coefficients: tuple of coefficients of the polinom, starts with the leading coefficient.
        Polinom(1, 2, 3) -> x^2 + 2x + 3

        self.coefficients: tuple of coefficients of the polinom, starts with the constant term.
        """
        try:
            for c in coeff:
                c += 0
        except :
            raise TypeError("coefficients must be numbers")
        coefficients = list(coefficients[::-1])
        while coefficients and coefficients[-1] == 0:
            coefficients.pop()
        self.coefficients = tuple(coefficients)



A `Polinom` osztály már a múlt órán is túl nagyra nőtt. Ilyen esetben célszerű a már letesztelt kódot külön fileba rakni és beolvasni. 

- Az egyik lehetőség egy telepíthető Python csomag készítése (erről itt nem lesz szó)

- Az egyszerűbb út az, hogy a file `.py` kiterjesztésű és a munkakönyvtárban van. Ezután 
```
import polinom
```

vagy
```
from polinom import *
```
sorokkal a `polinom.py` fileban definiált értékek elérhetőve válnak.


In [30]:
! head -n 15 polinom.py 

"""
Polinom osztály definíció.

... magyarázatok, stb
"""


__all__ = ["Polinom"]


def format_tag(exponent, coeff):
    sep = "+" if coeff >= 0 else "-"
    coeff = abs(coeff)
    if exponent == 0:
        return sep, f"{coeff}"


In [31]:
from polinom import *

In [32]:
[name for name in dir() if not name.startswith("_")]

['@py_builtins',
 '@pytest_ar',
 'In',
 'MyError',
 'Out',
 'Polinom',
 'SimpleWith',
 'Timer',
 'convert_to_int',
 'division',
 'exit',
 'f',
 'filepath',
 'get_ipython',
 'ipytest',
 'print_dunder',
 'print_lines',
 'print_methods',
 'quit',
 'sw',
 'time',
 'timer']

### Használjuk a Polinom osztályunkat valamire!

Számítsuk ki az első néhány Hermite polinomot! Definíció
$$
h_n(x) = (-1)^n\cdot e^{x^2/2}\left(\frac{d}{dx}\right)^n e^{-x^2/2}
$$
azaz:
$$
e^{-x^2/2} h_n(x) = (-1)^n\cdot \left(\frac{d}{dx}\right)^n e^{-x^2/2}
$$
Rekurzió a definíció alapján:
$$
\begin{aligned}
e^{-x^2/2} h_{n+1}(x) &= 
(-1)\cdot \frac{d}{dx}(e^{-x^2/2} h_n(x))
\\&=
(-1)\cdot e^{-x^2/2}(-xh_n(x) +h_n'(x))
\end{aligned}
$$
$$
    h_{n+1}(x) = xh_n(x) - h_n'(x)
$$



In [33]:
def hermite():
    h, x = Polinom(1), Polinom(1, 0)
    while True:
        yield h
        h = x*h-h.D()

In [34]:
for i, h_i in zip(range(5), hermite()):
    print(f"{i}. Hermite polinom: {h_i}")

0. Hermite polinom: 1
1. Hermite polinom: x
2. Hermite polinom: x^{2}-1
3. Hermite polinom: x^{3}-3x
4. Hermite polinom: x^{4}-6x^{2}+3


In [36]:
from itertools import pairwise, islice

for i, (h0, h1) in enumerate(pairwise(islice(hermite(), 5)), 1):
    print(f"{i}*h_{i-1}(x)={str(i*h0):<10} h'_{i}(x)={str(h1.D()):<10} h_{i}(x)={str(h1):<10}")


1*h_0(x)=1          h'_1(x)=1          h_1(x)=x         
2*h_1(x)=2x         h'_2(x)=2x         h_2(x)=x^{2}-1   
3*h_2(x)=3x^{2}-3   h'_3(x)=3x^{2}-3   h_3(x)=x^{3}-3x  
4*h_3(x)=4x^{3}-12x h'_4(x)=4x^{3}-12x h_4(x)=x^{4}-6x^{2}+3


- `islice(iterator, start, end)` egy sorozatból a `start` és `end indexek közötti elemeket veszi.
- `pairwise(iterator)` a párok sorozatát generálja.

HF. $h_n' = n h_{n-1}$.

Minden $n$-re van egy 1 főegyütthatós polinomunk $h_n$. Ekkor tetszőleges $p$ polinom felírható Hermite polinomok lineáris kombinációjaként.

Számítsuk ki az együthatókat.

In [37]:
def coefficients(p, basis):
    n = len(p)
    basis_polynomials = [q for q in islice(basis, n)]
    coeffs = [0]*n
    while len(p):
        c = p.leading_coeff() #/q.leading_coeff()
        exponent = len(p)-1
        p -= c*basis_polynomials[exponent] 
        coeffs[exponent] = c
    return coeffs

def polynomial(coeffs, basis):
    return sum(
        (c*p for c, p in zip(coeffs, basis)),
        start = Polinom()
    )

In [39]:
p = Polinom(2,0,1)
display(p)
coeffs = coefficients(p, hermite())
coeffs

Polinom(2, 0, 1)

[3, 0, 2]

In [40]:
for c_i, h_i in zip(coeffs, hermite()):
    display(c_i*h_i)

Polinom(3)

Polinom()

Polinom(2, 0, -2)

In [41]:
polynomial(coeffs, hermite())


Polinom(2, 0, 1)

## Faktoriális hatvány.

$$
\begin{aligned}
\text{hatvány}&&x^0 &= 1,& x^n &= x \cdot x^{n-1}\\
\text{faktoriális hatvány}&&x^{[0]} &= 1,& x^{[n]} &= x \cdot (x-1)^{[n-1]}
\end{aligned}
$$
<!-- 
$$
x^{[0]} = 1, \quad    x^{[n]} = x\cdot (x-1)^{[n-1]} 
$$
 -->
$$
x^{[n]} = x(x-1)\cdots (x-n+1)
$$

**HF**. A faktoriális hatványra is igaz a binomiális tétel!
$$
    (a+b)^{[n]} = \sum_{k} \binom{n}{k} a^{[k]} b^{[n-k]}
$$

In [42]:
def factorial_powers():
    n = 0
    p = Polinom(1) # kontans 1
    while True:
        yield p
        p *= Polinom(1, -n) # x-n
        n += 1

def nth_factorial_power(n):
    return next(islice(factorial_powers(), n, n+1))

In [43]:
for d in [2, 3, 5]:
    display(nth_factorial_power(d))

Polinom(1, -1, 0)

Polinom(1, -3, 2, 0)

Polinom(1, -10, 35, -50, 24, 0)

In [44]:
from itertools import accumulate # részlet összegek sorozata

p3 = nth_factorial_power(3)
[*accumulate(p3(x) for x in range(10))]

[0, 0, 0, 6, 30, 90, 210, 420, 756, 1260]

In [47]:
p4 = nth_factorial_power(4)
[p4(x) for x in range(0,11)]

[0, 0, 0, 0, 24, 120, 360, 840, 1680, 3024, 5040]

In [48]:
%%ipytest

def test_conjecture():
    cum_sum = 0
    for x in range(1000):
        cum_sum += p3(x)
        assert 4*cum_sum == p4(x+1)

.                                                                                            [100%]
1 passed in 0.01s


Kombinatorika gyakorlaton szerepelt a következő összefüggés:
$$
    \sum_{k=r}^n \binom{k}{r} = \binom{n+1}{r+1}
$$  

$(r+1)!$ rászorozva:
$$
   (r+1)\sum_{k=0}^n k^{[r]}  = (n+1)^{[r+1]}
$$
Ezt az összefüggést láttuk $r=3$-ra és $n=0,\dots,10$-re


A faktoriális hatványok minden $n$-re definiálnak egy 1 főegyütthatós, $n$-edfokú  polinomot, így 
$$
    x^r = \sum_{i=0}^{r} c_i x^{[i]}
$$

Ebből:
$$
\begin{aligned}
    (r+1)!\sum_{k=0}^n k^r
    &= (r+1)!\sum_{k=0}^{n} \sum_{i=0}^ r c_i k^{[i]} 
%     \\ &
    =
    \sum_{i=0}^r c_i (r+1)!\sum_{k=0}^n k^{[i]} 
    \\ &
    = \sum_{i=0}^{r} c_i \frac{(r+1)!}{i+1} (n+1)^{[i+1]} 
%     \\&
    = \sum_{i=0}^{r} c_i\frac{(r+1)!}{i+1} (n+1)\cdot(n^{[i]}) 
\end{aligned}
$$

In [49]:
for r in range(5):
    p = Polinom(1, 0)**r # x^r
    coeffs = coefficients(p, factorial_powers())
    print(f"{p=!s}, {coeffs=}, {polynomial(coeffs, factorial_powers())=!s}")


p=1, coeffs=[1], polynomial(coeffs, factorial_powers())=1
p=x, coeffs=[0, 1], polynomial(coeffs, factorial_powers())=x
p=x^{2}, coeffs=[0, 1, 1], polynomial(coeffs, factorial_powers())=x^{2}
p=x^{3}, coeffs=[0, 1, 3, 1], polynomial(coeffs, factorial_powers())=x^{3}
p=x^{4}, coeffs=[0, 1, 7, 6, 1], polynomial(coeffs, factorial_powers())=x^{4}


In [50]:
import math 

def power_sum_polynomial(r):
    """ 
    returns (r+1)!*p, such that p(n) = sum_{k=0}^n k^r
    """
    coeffs = coefficients(Polinom(1, 0)**r, factorial_powers())
    factorial = math.factorial(r+1)
    coeffs = [c*factorial//(i+1) for i, c in enumerate(coeffs)]
    return polynomial(coeffs, (Polinom(1, 1)*p for p in factorial_powers()))


In [51]:
power_sum_polynomial(5) # 6!-szor az 5. hatványösszeg polinom.

Polinom(120, 360, 300, 0, -60, 0, 0)

### Az első néhány hatványösszeg polinom

In [52]:
for r in range(1, 5):
    display(power_sum_polynomial(r))

Polinom(1, 1, 0)

Polinom(2, 3, 1, 0)

Polinom(6, 12, 6, 0, 0)

Polinom(24, 60, 40, 0, -4, 0)

HF. Az első néhány polinomnak racionális gyökei vannak. Igaz-e ez az összesre?

In [53]:
def mk_power_sum(r):
    p = power_sum_polynomial(r)
    factorial = math.factorial(r+1)
    def power_sum(n):
        return p(n)//factorial
    return power_sum

In [55]:
import ipytest
ipytest.autoconfig()

In [56]:
%%ipytest
import random 

def test_power_sum():
    for r in range(5):
        p = mk_power_sum(r)
        for _ in range(20):
            n = random.randint(0, 1000)
            assert p(n) == sum(x**r for x in range(n+1))

.                                                                                            [100%]
1 passed in 0.02s


### Polinomok kompozíciója.

Ha a számokat a konstans Polinomokkal azonosítjuk, akkor Polinomot Polinom helyen is kiértékelhetünk.
Ehhez az összeadást módosítottam:

```python

def __add__(self, other):
    if not isinstance(other, Polinom):
        other = Polinom(other)
    ...
```

In [57]:
from polinom import *
p, q  = Polinom(2, 3, 0, 1), Polinom(1, 0, 1)
display(p)
display(q)
p(q)

Polinom(2, 3, 0, 1)

Polinom(1, 0, 1)

Polinom(2, 0, 9, 0, 12, 0, 6)

### Szükséges-e a Polinom osztályt megírni?

- Ha van egy jól használható típus, pl. polinom, permutáció, stb. akkor azt  nagy valószínűséggel valaki már megírta, letesztelte.

- Polinom osztály pl. a `numpy` könyvtárban is  van, permutáció pedig a  `sympy` könytárban.


In [ ]:
import numpy as np

np.polynomial.Polynomial?

In [ ]:
from sympy.combinatorics import Permutation

Permutation?

Csebisev polinomok, Hermite polinomok.

Hatványösszeg. fractions könyvtár, Fraction típus.

# `contextmanager` generátor függvényből

Láttunk példát arra, hogyan tudunk osztályként `contextmanager`-t definálini.

Van egy egyszerűbb út is. `contextmanager`-t definiálhatunk generátor függvényből is.

In [58]:
from contextlib import contextmanager

@contextmanager
def simple_context(value):
    try:
        print("before yield")
        yield value
        print("after yield")
    finally:
        print("finally")

In [59]:
with simple_context("1") as x:
    print(f"inside {x=}")
    int(x)

before yield
inside x='1'
after yield
finally


A `contextmanager` dekorátor nagyjából az alábbi dolgot csinálja:

In [60]:
def simplified_cm(generator_function):
    
    class cm:
        def __init__(self, *args, **kwargs):
            self.it = generator_function(*args, **kwargs)
            
        def __enter__(self):
            return next(self.it)
        
        def __exit__(self, exc_type, *args):
            ## hiba kezelés
            if exc_type is None:
                return next(self.it, None)
             
   
    return cm

In [61]:
@simplified_cm
def simple_context(value):
    try:
        print("before yield")
        yield value
        print("after yield before finally")
    finally:
        print("finally")


In [63]:
with simple_context('1') as x:
    print(f"inside {x=}")
    int(x)

before yield
inside x='1'
after yield before finally
finally


In [64]:
with simple_context('x') as x:
    print(f"inside {x=}")
    int(x)

before yield
inside x='x'
finally


ValueError: invalid literal for int() with base 10: 'x'

**HF** Írjuk meg a `Timer` contextmanager-t generátor függvénnyel:

```python
from contextlib import contextmanager

@contextmanager
def timer():
    ...
```

## További hasznos beépített könyvtárak 

- `itertools`. Hasznos iterátorok.

- `functools`. Függvény manipulációk

- `math`, `cmath`. matematikai függvények, komplex függvények

- `fractions`. Törtek

Írás, olvasás különböző file formátumokba

- `json`

- `yaml`

- `sqlite3`

- `csv`, ehelyett inkább a `pandas` könyvtárat használja az ember.



## Itertools

A használathoz importálni kell a könyvtárat:

In [68]:
import itertools

print_methods(itertools, per_line=6)
help(itertools.zip_longest)

accumulate, chain, combinations, combinations_with_replacement, compress, count
cycle, dropwhile, filterfalse, groupby, islice, pairwise
permutations, product, repeat, starmap, takewhile, tee
zip_longest
Help on class zip_longest in module itertools:

class zip_longest(builtins.object)
 |  zip_longest(iter1 [,iter2 [...]], [fillvalue=None]) --> zip_longest object
 |  
 |  Return a zip_longest object whose .__next__() method returns a tuple where
 |  the i-th element comes from the i-th iterable argument.  The .__next__()
 |  method continues until the longest iterable in the argument sequence
 |  is exhausted and then it raises StopIteration.  When the shorter iterables
 |  are exhausted, the fillvalue is substituted in their place.  The fillvalue
 |  defaults to None or can be specified by a keyword argument.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __ne

Néhány hasznos iterátor:

- `itertools.product` Két (vagy több) sorozat Decartes szorzatán megy végig.

- `itertools.combinations` Egy sorozat $r$ elemű ismétlés nélküli kombinációin megy végig.

- `itertools.permutations`. Egy sorozat permutációin megy végig.

- `itertools.pairwise`. Az egymást követő párokon megy végig.

### Példák

In [69]:
[*itertools.product(range(3), repeat=2)]

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [70]:
[*itertools.combinations(range(4), r=3)]

[(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]

In [71]:
[*itertools.permutations(range(1, 4))]

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [72]:
[*itertools.pairwise(range(4))]

[(0, 1), (1, 2), (2, 3)]

- `itertools.accumulate`. Alapértelmezésben egy sorozat részletösszegeinek a sorozatát állítja elő.

In [74]:
[*itertools.accumulate(range(1, 5), initial=0)]

[0, 1, 3, 6, 10]

Megadható kezdeti érték is, ekkor ez lesz a sorozat első eleme.

In [75]:
[*itertools.accumulate(range(1, 5), initial=0)]

[0, 1, 3, 6, 10]

Az is megadható, hogy miként akkumuláljuk az értéket.

In [76]:
[*itertools.accumulate(range(1, 5), func=lambda a, b: a+2*b-1, initial=0)]

[0, 1, 4, 9, 16]

- `itertools.chain`. iterátorokat fűz egymás után.
- `itertools.chain.from_iterable` egy olyan iterátor kap, aminek az elemei is iterátorok.

In [77]:
[*itertools.chain(range(3), range(15,18))]

[0, 1, 2, 15, 16, 17]

In [82]:
[*itertools.chain( *  (range(i) for i in range(1, 4)) )] 

[0, 0, 1, 0, 1, 2]

In [80]:
[*itertools.chain.from_iterable((range(i) for i in range(1, 4)))]

[0, 0, 1, 0, 1, 2]

- `count` olyan iterátor ami a természetes számok sorozatát szolgáltatja

In [83]:
it = itertools.count()
print(f"{next(it)=}, {next(it)=}, {next(it)=}")

next(it)=0, next(it)=1, next(it)=2


- `cycle` egy adott sorozat elemeit ismétli, ha a végére ér kezdi előlről.

In [85]:
it = itertools.cycle(map(str.upper, "alma"))
for _ in range(17):
    print(next(it), end=" ")

A L M A A L M A A L M A A L M A A 

- `repeat` egy elemet ismétel

In [86]:
[*itertools.repeat("alma", 4)]

['alma', 'alma', 'alma', 'alma']

- `dropwhile` kidobja a sorozat első néhány elemét, amíg a feltétel igaz.
- `takewhile` Addig megy el amíg  a feltétel igaz
- `filterfalse` A hamis értékeket tartja meg.
- `islice` Ha a sorozat egy lista, akkor a slice indexelésnek felel meg.

In [87]:
[*itertools.dropwhile(lambda x: x, [1, -1, 0, 1, 2, 0])]

[0, 1, 2, 0]

In [88]:
[*itertools.takewhile(lambda x: x, [1, -1, 0, 1, 2, 0])]

[1, -1]

In [89]:
[*itertools.filterfalse(lambda x: x, [1, -1, 0, 1, 2, 0])]

[0, 0]

In [90]:
[*itertools.islice(range(10), 2, 7, 2)]

[2, 4, 6]

**Feladat**: van egy [szám](https://en.wikipedia.org/wiki/Champernowne_constant), ami így van definiálva: 0.12345678910111213141516.... Mi ennek a számnak az egymilliomodik számjegye? (Az első számjegy $0$, a második $1$, stb.)

Ötlet: ahogy jönnek a számok egymás után (0, 1, 2, ..., 10, 11, 12, ..., 100, 101, 102, ...), mikor érjük el az $n$-edik számjegyet?

In [91]:
# Imperatív stílusú megoldás
def calc_digit(num_digits):
    num_digits -= 1 # zero indexing
    n = 0
    while len(s:= str(n)) <= num_digits:
        num_digits -= len(s)
        n += 1
    return int(s[num_digits])

Hogy oldottuk meg?

* bevezettük egy $n$ **változót** az egész számokra, amelyet folyamatosan növeltünk,
* egy **while ciklus**ban iteráltunk, amíg el nem értük a keresett számot,
* egy **változóban** tároltuk, hogy a maradék sorozat hányadik jegyét keressük,
* az utolsó számból kiolvastuk a keresett számjegyet.

In [92]:
%%ipytest

def test_calc_digit():
    for i in range(1, 11):
        assert calc_digit(i) == i-1


.                                                                                            [100%]
1 passed in 0.01s


In [93]:
calc_digit(10**6)

4

In [94]:
# Funkcionális stílusú megoldás
import itertools

def calc_digit_functional(num_digits):
    digits = itertools.chain.from_iterable(
        map(str,
            itertools.count() # 0, 1, 2, 3, ...
           ) # "0", "1", ... , "10", "11",... 
    ) # "0", "1", ..., "1", "0", ...
    result = next(itertools.islice(digits, num_digits-1, num_digits))
    return int(result)

calc_digit_functional(10**6)

4

Itt nincs `for`/`while` ciklus, vagy `if`. Generáltunk egy sorozatot, arra alkalmaztunk műveleteket, és kiolvastuk az eredményt.

**HF** Hatékonyabb megoldás. 

#### Run-length-encoding

**HF** mit csinál a `itertools.groupby` függvény?

In [95]:
def rle(seq):
    for x, g in itertools.groupby(seq):
        yield x, len(list(g))

In [96]:
[*rle('aabbcde')]

[('a', 2), ('b', 2), ('c', 1), ('d', 1), ('e', 1)]

# `functools` könyvtár

In [ ]:
import functools
print_methods(functools, per_line=5)

Leggyakrabban használt függvények:

- `reduce`
- `lru_cache`
- `wraps`
- `partial`

## Példák.

#### `reduce` függvény.

In [ ]:
functools.reduce(lambda x, y: x+y, range(5), 0) # (((((0 + 0) + 1) + 2) + 3) + 4) 

```python
def reduce(f, seq, initial):
    a = initial 
    for b in seq:
        a = f(a, b)
    return a
```

### `lru_cache` dekorátor

In [ ]:
# @functools.lru_cache
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

with Timer():
    f = fibonacci(30)

print(f)

### `wraps` dekorátor

In [ ]:
def show_call(fun):
    @functools.wraps(fun)
    def f(*args):
        result = fun(*args) 
        print(f"{fun.__name}({', '.join(map(str, args))})")
        return result
    return f

@show_call
def add(a: int, b: int) -> int:
    """adds two number"""
    return a+b 

help(add)

### `partial` 

In [ ]:
def f(a, b):
    return a+b

inc1 = functools.partial(f, b=1)
inc1(2)

**HF** Mi történik a következő kódrészletben? 

In [ ]:
import sys
import textwrap
from functools import wraps
from contextlib import contextmanager

def add_indent(write, indentation):
    @wraps(write)
    def f(string):
        return write(textwrap.indent(string, indentation))
    return f

@contextmanager
def indented(indentation):
    try:
        sys.stdout.write = add_indent(sys.stdout.write, indentation)
        yield
    finally:
        sys.stdout.write = sys.stdout.write.__wrapped__

In [ ]:
with indented("|---"):
    print("alma\nkorte")
    with indented("-> "):
        print("alma\nkorte")
    print("alma\nkorte")
    

print("alma\nkorte")

# Néhány szöveges fileformátum

`json` és `yaml`

- `json` ,,javascript object notation''. Hasonló ahhoz, amit `list`, `dict` esetén a képernyőn látunk. 
  Nem lehet mindent típust `json`-fileba menteni, de a legfontosabbakat igen:
   
  * `int`, `float`, `str`, `bool`
  * `list`, `dict`.

  `json` fileban nincs kommentelési lehetőség.

- `yaml`. Több típust tud kezelni, ezért alapból nem `safe`, de könnyen olvasható, szerkeszthető, lehetnek kommentek is. 

In [ ]:
import json

data = [
    {'a': 1, 'b':True, 'pi': .314, "nested": {"c": 3}},
    23,
    [1, 2, 3]
]

json.dumps(data)

In [ ]:
# fileba írás
with open("/tmp/test.json", "w") as f:
    json.dump(data, f)

In [ ]:
with open("/tmp/test.json", "r") as f:
    print(json.load(f))

In [ ]:
import yaml

print(yaml.dump(data))
# vagy
with open("/tmp/test.yaml", "w") as f:
    yaml.dump(data, f)
    
with open("/tmp/test.yaml", "r") as f:
    print(yaml.safe_load(f))